In [1]:
import math
import pygmt
import imageio
import xarray as xr
import numpy as np
import pyvista as pv
from scripts import stratal as strat
from pyevtk.hl import gridToVTK

import matplotlib
from matplotlib import cm
import cmocean as cmo
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.ndimage import gaussian_filter

label_size = 7
matplotlib.rcParams['xtick.labelsize'] = label_size
matplotlib.rcParams['ytick.labelsize'] = label_size
matplotlib.rc('font', size=6)

%matplotlib inline
# %config InlineBackend.figure_format = 'svg'

# Extract stratigraphy

We will now look at the recorded stratigraphy. The stratigraphic layer are recorded in gospl as `HDF5` files stored in the output folder as `stratal.XX.pX.h5` where `XX` is the output step and `X` the processor number.


### Stratal record definition

The following information are stored:

+ elevation at time of deposition, considered to be to the current elevation for the top stratigraphic layer `stratZ`.
+ thickness of each stratigrapic layer `stratH` accounting for both erosion & deposition events.
+ porosity of coarse sediment `phiS` in each stratigraphic layer computed at center of each layer.

We will use the `stratal.py` Python class to extract the information above. It requires the following arguments:

+ `path`: the path to the input file
+ `filename`: the name of the input file
+ `layer`: the stratal file you wish to output

In [2]:
# We pick the last step
step = 35 

# Load the function and specify the input file
strati = strat.stratal(path='./', filename='kup_strat.yml', layer=step, model='utm')

# Read the stratigraphic dataset
strati.readStratalData()

# Interpolate the spherical dataset on a UTM regular grid
# by specifying the desired resolution and interpolation neighbours
strati.buildUTMmesh(res=2500., nghb=3)

Created sedimentary layers: 35
Number of sedimentary layers: 349
Start building regular stratigraphic arrays
Percentage of arrays built : [####################] 100.0% DONE


### Extracting the map for the region

In [10]:
# Smoothing values
sigma=0.1

We now specify the stratigraphic values to extract 

In [11]:
xo = strati.x.min()
xm = strati.x.max()
yo = strati.y.min()
ym = strati.y.max()
lon = np.linspace(xo, xm, strati.nx)
lat = np.linspace(yo, ym, strati.ny)

lons = [0, strati.nx]
lats = [0, strati.ny]
idlat1 = 0
idlat2 = len(lat)
idlon1 = 0
idlon2 = len(lon)

shape = (strati.nx, strati.ny, strati.curLay)
    
# Define the variables shape    
# shape = (idlon2-idlon1,idlat2-idlat1,strati.curLay)

# lon
x = np.empty(shape)
# lat
y = np.empty(shape)
# elevation
z = np.empty(shape)
# elevation at the time of deposition
e = np.empty(shape)
# stratal thickness
h = np.empty(shape)
# layer ID
t = np.empty(shape)
# porosity
ps = np.empty(shape)

# Now let's define the surface elevation
zz = strati.zi[-1, idlat1:idlat2, idlon1:idlon2]
zz = gaussian_filter(zz, sigma)

We use `numba` to speed things up, the function `getVals` will extract the different variables for individual stratigraphic layer.

In [12]:
import numba as nb

@nb.jit(nopython=True)
def getVals(k, idlat1, idlat2, idlon1, idlon2, lon, lat, zz, zi, thu, th, phiSi, topz):
    
    shape = (idlon2-idlon1,idlat2-idlat1)
    lx = np.empty(shape)
    ly = np.empty(shape) 
    lz = np.empty(shape)  
    le = np.empty(shape) 
    lh = np.empty(shape)  
    lps = np.empty(shape)  
    lt = np.empty(shape) 
    
    for j in range(shape[1]):
        for i in range(shape[0]):
            lx[i, j] = lon[i+idlon1]
            ly[i, j] = lat[j+idlat1]  
            if topz is None:
                lz[i, j] = zz[j, i]
            else:
                lz[i, j] = topz[i, j] - thu[j, i]
            le[i, j] = zi[j, i]
            lh[i, j] = th[j, i]
            lps[i, j] = phiSi[j, i]
            lt[i, j] = k
    
    return lx, ly, lz, le, lh, lps, lt

Looping over each layer we extract the information for the desired region:

In [ ]:
for k in range(strati.curLay - 1, -1, -1):
    
    if sigma > 0:
        th = gaussian_filter(strati.thi[k, idlat1:idlat2, idlon1:idlon2], sigma)
    th[th < 0] = 0.0
    if k < strati.curLay - 1:
        if sigma > 0:
            thu = gaussian_filter(strati.thi[k + 1, idlat1:idlat2, idlon1:idlon2], sigma)
        thu[thu < 0] = 0.0
    else:
        thu = None
    zi = strati.zi[k, idlat1:idlat2, idlon1:idlon2]
    phiSi = strati.phiSi[k, idlat1:idlat2, idlon1:idlon2]
    
    if k == strati.curLay - 1:
        topz = None
    else:
        topz = z[:, :, k + 1] 
        
    ( x[:, :, k], y[:, :, k], z[:, :, k], 
     e[:, :, k], h[:, :, k], ps[:, :, k], 
     t[:, :, k] 
    ) = getVals(k, idlat1, idlat2, idlon1, idlon2,
                lon, lat, zz, zi, thu, th, phiSi, topz)

We save the dataset as a `vtk` file:

In [ ]:
gridToVTK(
            'strati_'+str(step),
            x,
            y,
            z,
            pointData={"dep elev": e, "th": h, "layID": t, "phiC": ps,},
        )